In [1]:
import pandas as pd
import numpy as np
import os
from math import pi
from bokeh.io import show
from bokeh.models import LogColorMapper, BasicTicker, PrintfTickFormatter, ColorBar
from bokeh import plotting
from bokeh.palettes import Viridis256, magma
plotting.output_notebook()

Loading BokehJS ...

In [2]:
h5_dir = '/users/lukestarnes/Documents/adsb'
h5_files = [os.path.join(h5_dir, f) for f in os.listdir(h5_dir) if f.endswith('.h5')]
h5_file = h5_files[2]

In [3]:
%%time
row_len = 200_000
pickle_name = f'{os.path.basename(h5_file)}-{row_len}.p'
pickle_path = os.path.join(os.getcwd(), 'data', pickle_name)
print(pickle_path)
if os.path.exists(pickle_path):
    df = pd.read_pickle(pickle_path)
else:
    with pd.HDFStore(h5_file) as store:
        df = store.select('data', stop = row_len)
        df.to_pickle(pickle_path)

/Users/lukestarnes/Documents/General Stuff/Software/Dev/GitHub/sfte2018-adsb/data/2018-06-24.h5-200000.p
CPU times: user 506 ms, sys: 221 ms, total: 727 ms
Wall time: 762 ms


In [4]:
df.loc[df.Year.isin(['Year','','2104','2105']),'Year'] = np.nan

In [5]:
cou_vs_year = df.groupby(['Cou', 'Year']).count()['Icao'].unstack().fillna(0)
cou_vs_year.index.name = 'Country'
cou_vs_year.columns.name = 'Year'
cou_vs_year = cou_vs_year[cou_vs_year.columns[35:]]
cou_vs_year.loc[cou_vs_year.sum(axis=1).sort_values(ascending=False).index[:40], :].sort_index(ascending=True)
cou_vs_year_percent = cou_vs_year.div(cou_vs_year.sum(axis=1),axis=0) * 100
year_vs_cou_percent = cou_vs_year_percent.T
year_vs_cou = cou_vs_year.T

In [7]:
year_vs_cou_percent.index = year_vs_cou_percent.index.astype(str)

years = list(year_vs_cou_percent.index)
months = list(year_vs_cou_percent.columns)

arr_p = pd.DataFrame(year_vs_cou_percent.stack(), columns=['Percentage']).reset_index()
arr_c = pd.DataFrame(year_vs_cou.stack(), columns=['Count']).reset_index()
total_by_cou = year_vs_cou.sum(axis=0)
arr_c['Total'] = arr_c.Country.map(total_by_cou)
arr = pd.merge(arr_p, arr_c, how='outer', left_on=['Year', 'Country'], right_on=['Year', 'Country'])

colors = magma(256)
mapper = LogColorMapper(palette=colors, low=0, high=100)

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = plotting.figure(title="Aircraft Registered to Country by Year ({0} - {1})".format(years[0], years[-1]),
           x_range=years, y_range=list(reversed(months)),
           x_axis_location="above", plot_width=950, plot_height=1200,
           tools=TOOLS, toolbar_location='above',
           tooltips=[('Country', '@Country'), 
                     ('Year', '@Year'), 
                     ('Aircraft Count', '@Count'),
                     ('Percent of Fleet', '@Percentage% of fleet'),
                     ("Total Aircraft", '@Total')])

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "8pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 3

p.rect(x="Year", y="Country", width=1, height=1,
       source=arr,
       fill_color={'field': 'Percentage', 'transform': mapper},
       line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="8pt",
                     ticker=BasicTicker(desired_num_ticks=10),
                     label_standoff=6, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')
show(p)